In [16]:
! pip install pyodbc
! pip install sqlalchemy

import pandas as pd
import json
import pyodbc
import sqlalchemy as db
import urllib


In [17]:
#Inseram tabelul "school_details" si facem un dataframe

In [53]:
file_path = "school_details.json"

with open(file_path, encoding="utf8") as json_file:
    school_details = json.load(json_file)
    
data = pd.DataFrame(school_details)
data

,idSchool,internalIdSchool,idParentSchool,schoolYearDescription,code,siruesCode,longName,shortName,schoolType,statut,...,streetNumber,postalCode,phoneNumber,faxNumber,email,idSchoolYear.code,schoolNumbers.idSchool,schoolNumbers.studyFormationsCount,schoolNumbers.studentsCount,schoolNumbers.personnelCount
0,11269910,10131270,None,Anul şcolar 2018-2019,261200154,23054595,"ASOCIAŢIA CREŞTINĂ DE CARITATE ""GOSEN"" ARAD/ G...","GRĂDINIŢA ""GOSEN"" ARAD",Unitate de învățământ,Cu personalitate juridică,...,121-125,310365,257272477,257272477,gradinitagosen@yahoo.com,2018-2019,11269910,4,131,7
1,11256876,11192270,None,Anul şcolar 2018-2019,1061206943,None,ASOCIAȚIA PENTRU PROMOVAREA VALORILOR AUTENTIC...,ASOCIAȚIA VALORILOR BZ,Unitate de învățământ,Cu personalitate juridică,...,4,125300,762724000,238563970,secretariat@cnpaulescu.ro,2018-2019,11256876,0,0,0
2,11256865,11192168,None,Anul şcolar 2018-2019,1061206875,None,ASOCIAȚIA UMANITARĂ EDUSTAR BUZĂU,ASOCIATIA EDUSTAR,Unitate de învățământ,Cu personalitate juridică,...,14,120010,773826330,238725913,luminitamusat2006@yahoo.com,2018-2019,11256865,0,0,0
3,11284187,11284187,11270069,Anul şcolar 2018-2019,5162103686,None,CABINET INTERSCOLAR DE ASISTENTA PSIHOPEDAGOGI...,CIAP LICEUL PEDADOGIC STEFAN BANULESCU,Unitate de învățământ,Arondată,...,1,None,242317042,242317042,None,2018-2019,11284187,0,0,0
4,11270113,10049611,11270069,Anul şcolar 2018-2019,5142100248,None,CABINET INTERSCOLAR DE ASISTENTA PSIHOPEDAGOGI...,CIAP LICEUL NR1 BORCEA_SCOALA PERISORU,Centru județean de resurse și asistență educaț...,Arondată,...,10B,None,242317042,None,None,2018-2019,11270113,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18910,11271997,11045085,11271994,Anul şcolar 2018-2019,562108182,362678,ŞCOALĂ PRIMARĂ NR.1 LUPOAIA,ŞCOALĂ PRIMARĂ NR.1 LUPOAIA,Unitate de învățământ,Arondată,...,163,417286,None,None,None,2018-2019,11271997,5,0,0
18911,11262566,10127944,11262530,Anul şcolar 2018-2019,1562102445,None,ȘCOALĂ PRIMARĂ NR.2 I. L. CARAGIALE,ȘCOALĂ PRIMARĂ NR.2 I. L. CARAGIALE,Unitate de învățământ,Arondată,...,1,137255,None,None,None,2018-2019,11262566,4,0,0
18912,11277287,11277287,11268974,Anul şcolar 2018-2019,1962105989,None,"ȘCOALĂ PRIMARĂ ""TAMÁSI ÁRON"" ODORHEIU SECUIESC","S04 ""TAMASI ARON"" ODORHEI",Unitate de învățământ,Arondată,...,4,535600,266218379,266218379,lt.tamasiaron@eduhr.ro,2018-2019,11277287,1,0,0
18913,11260231,11142774,None,Anul şcolar 2018-2019,2261207102,None,"ŞOALA PRIMARĂ ""MARIA MONTESSORI"", IAŞI",SC PRI MARIA MONTESSORI IASI,Unitate de învățământ,Cu personalitate juridică,...,6,None,232270240,232270240,inscrieri@gradinita-tesori.ro,2018-2019,11260231,5,32,8


In [54]:
#Ne uitam peste coloane si le eliminam pe cele pe care nu le folosim in analiza

In [55]:
data.columns

Index(['idSchool', 'internalIdSchool', 'idParentSchool',
       'schoolYearDescription', 'code', 'siruesCode', 'longName', 'shortName',
       'schoolType', 'statut', 'isPj', 'fiscalCode', 'operatingMode',
       'propertyForm', 'fundingForm', 'county', 'locality', 'street',
       'streetNumber', 'postalCode', 'phoneNumber', 'faxNumber', 'email',
       'idSchoolYear.code', 'schoolNumbers.idSchool',
       'schoolNumbers.studyFormationsCount', 'schoolNumbers.studentsCount',
       'schoolNumbers.personnelCount'],
      dtype='object')

In [56]:
data.drop(columns = ['internalIdSchool', 'idParentSchool',
       'schoolYearDescription', 'code', 'siruesCode', 'longName', 'shortName',
       'schoolType', 'statut', 'isPj', 'fiscalCode', 'operatingMode',
       'propertyForm', 'fundingForm', 'locality', 'street',
       'streetNumber', 'postalCode', 'phoneNumber', 'faxNumber', 'email',
       'idSchoolYear.code', 'schoolNumbers.idSchool',
       'schoolNumbers.studyFormationsCount'], inplace = True)

In [57]:
#Ne asiguram ca baza de date nu are valori lipsa

In [58]:
data.isnull().sum()

idSchool                        0
county                          0
schoolNumbers.studentsCount     0
schoolNumbers.personnelCount    0
dtype: int64

In [59]:
#Ne asiguram ca baza de date nu are duplicate

In [60]:
duplicates =data[data.duplicated()]
print(duplicates)

Empty DataFrame
Columns: [idSchool, county, schoolNumbers.studentsCount, schoolNumbers.personnelCount]
Index: []


In [61]:
#Schimbam denumirile variabilelor "county", "schoolNumbers.studentsCount", "schoolNumbers.personnelCount" pentru a fi mai usor prelucrarea lor

In [62]:
data = data.rename(columns = {"county": "Judet"})
data = data.rename(columns = {"schoolNumbers.studentsCount": "Numar_elevi"})
data = data.rename(columns = {"schoolNumbers.personnelCount": "Numar_personal_liceu"})

In [63]:
#Inlocuim diacriticele din variabila "Judet"

In [65]:
diacr = {"Ă":"A", "Ş":"S","Ţ":"T","Â":"A","Î":"I"}
translation_table = str.maketrans(diacr)
data["Judet"] = data["Judet"].str.translate(translation_table)

data

,idSchool,Judet,Numar_elevi,Numar_personal_liceu
0,11269910,ARAD,131,7
1,11256876,BUZAU,0,0
2,11256865,BUZAU,0,0
3,11284187,CALARASI,0,0
4,11270113,CALARASI,0,0
...,...,...,...,...
18910,11271997,BIHOR,0,0
18911,11262566,DAMBOVITA,0,0
18912,11277287,HARGHITA,0,0
18913,11260231,IASI,32,8


In [66]:
#Observam ca avem judetele scrise cu litera mare, astfel incercam sa le normalizam

In [67]:
data["Judet"] = data["Judet"].apply(lambda x:x[0]+x[1:].lower())
data

,idSchool,Judet,Numar_elevi,Numar_personal_liceu
0,11269910,Arad,131,7
1,11256876,Buzau,0,0
2,11256865,Buzau,0,0
3,11284187,Calarasi,0,0
4,11270113,Calarasi,0,0
...,...,...,...,...
18910,11271997,Bihor,0,0
18911,11262566,Dambovita,0,0
18912,11277287,Harghita,0,0
18913,11260231,Iasi,32,8


In [68]:
data.to_csv("School_Details_Clean_Team4.csv", index = False)

In [69]:
#Inseram tabelul "tempo_jud"

In [70]:
file_path = "tempo_jud.json"

with open(file_path) as json_file:
    tempo_jud = json.load(json_file)
    
data1 = pd.DataFrame(tempo_jud)
data1

,Judet,Elevi_cu_BAC,Ateliere_scolare,Laboratoare,Echipamente,Terenuri_sport,Imigranti
0,Bihor,2915,108,253,6137,73,300
1,Bistrita-Nasaud,1553,47,104,3814,21,177
2,Cluj,3653,112,368,14700,75,692
3,Maramures,2449,67,135,6815,37,326
4,Satu Mare,1460,74,139,3722,39,205
5,Salaj,1171,51,134,2122,17,66
6,Alba,1817,82,162,4882,37,215
7,Brasov,2668,97,203,4005,40,634
8,Covasna,847,93,70,1842,16,66
9,Harghita,1422,99,134,3795,27,93


In [71]:
#Ne asiguram ca baza de date nu are valori lipsa

In [72]:
data1.isnull().sum()

Judet               0
Elevi_cu_BAC        0
Ateliere_scolare    0
Laboratoare         0
Echipamente         0
Terenuri_sport      0
Imigranti           0
dtype: int64